In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# install pyspark
!pip3 install pyspark==3.2.0

# install graphframes
!pip3 install graphframes

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=32be791c393da838612d511b653607cd3087b7bea1dc4051efe2afd305e27124
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 154 kB 5.1 MB/s 


In [ ]:
!cp -v /content/graphframes-0.8.2-spark3.2-s_2.12.jar $SPARK_HOME/jars/

'/content/graphframes-0.8.2-spark3.2-s_2.12.jar' -> '/content/spark-3.2.0-bin-hadoop3.2/jars/graphframes-0.8.2-spark3.2-s_2.12.jar'


In [ ]:
#import the packages
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import pandas as pd

findspark.init()

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# access the postgresql server
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")

In [ ]:
cursor = conn.cursor()

In [ ]:
# Selecting for officers with LESS than average number of award types 
edges_query = '''--Co-accusals of officers with DAM allegations earning less than average distinct award types
DROP TABLE IF EXISTS award_count;
CREATE TEMP TABLE award_count AS (
    SELECT officer_id offid, count(award_type) num_awards
    FROM data_award
    WHERE data_award.officer_id IN
                  (SELECT id
                  FROM data_officer
                  WHERE data_officer.id IN
                    (SELECT officer_id
                    FROM data_officerallegation
                    WHERE data_officerallegation.allegation_category_id IN
                      (SELECT id
                      FROM data_allegationcategory
                      WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
                      OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E'))))
    group by officer_id);

DROP TABLE IF EXISTS da_category_ids;
CREATE TEMP TABLE da_category_ids AS (
    SELECT id
    FROM data_allegationcategory
    WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
    OR data_allegationcategory.category_code IN ('08J', '024', '003', '003A', '003B', '003C', '003D', '003E'));

DROP TABLE IF EXISTS da_cohort;
CREATE TEMP TABLE da_cohort AS (
    SELECT DISTINCT officer_id
    FROM data_officerallegation
    WHERE allegation_category_id IN (SELECT * from da_category_ids));

DELETE FROM da_cohort c
USING award_count a WHERE a.offid = c.officer_id
AND a.num_awards > (SELECT AVG(num_awards) FROM award_count);

DROP TABLE IF EXISTS class_example;
CREATE TEMP TABLE class_example AS (
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC);

DROP TABLE IF EXISTS edgesq2;
CREATE TEMP TABLE edgesq2 AS (
SELECT src,dst,relationship FROM class_example
JOIN da_cohort ON da_cohort.officer_id = class_example.src);

select * from edgesq2;
'''

In [ ]:
# Selecting for officers with LESS than average number of award types 
nodes_query = '''--Co-accusals of officers with DAM allegations earning less than average distinct award types
DROP TABLE IF EXISTS award_count;
CREATE TEMP TABLE award_count AS (
    SELECT officer_id offid, count(award_type) num_awards
    FROM data_award
    WHERE data_award.officer_id IN
                  (SELECT id
                  FROM data_officer
                  WHERE data_officer.id IN
                    (SELECT officer_id
                    FROM data_officerallegation
                    WHERE data_officerallegation.allegation_category_id IN
                      (SELECT id
                      FROM data_allegationcategory
                      WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
                      OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E'))))
    group by officer_id);

DROP TABLE IF EXISTS da_category_ids;
CREATE TEMP TABLE da_category_ids AS (
    SELECT id
    FROM data_allegationcategory
    WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
    OR data_allegationcategory.category_code IN ('08J', '024', '003', '003A', '003B', '003C', '003D', '003E'));

DROP TABLE IF EXISTS da_cohort;
CREATE TEMP TABLE da_cohort AS (
    SELECT DISTINCT officer_id
    FROM data_officerallegation
    WHERE allegation_category_id IN (SELECT * from da_category_ids));

DELETE FROM da_cohort c
USING award_count a WHERE a.offid = c.officer_id
AND a.num_awards > (SELECT AVG(num_awards) FROM award_count);

DROP TABLE IF EXISTS class_example;
CREATE TEMP TABLE class_example AS (
SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship
FROM data_officerallegation da1
JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id
GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC);

DROP TABLE IF EXISTS edgesq2;
CREATE TEMP TABLE edgesq2 AS (
SELECT src,dst,relationship FROM class_example
JOIN da_cohort ON da_cohort.officer_id = class_example.src);

select * from edgesq2;

DROP TABLE IF EXISTS withoutRelation;
CREATE TEMP TABLE withoutRelation AS (
select id,first_name || ' ' || last_name officer_name from data_officer where id in
(select src from edgesq2) and id in (select dst from edgesq2));

DROP TABLE IF EXISTS award_count;
CREATE TEMP TABLE award_count AS (
    SELECT officer_id offid, count(award_type) num_awards
    FROM data_award
    WHERE data_award.officer_id IN
                  (SELECT id
                  FROM data_officer
                  WHERE data_officer.id IN
                    (SELECT officer_id
                    FROM data_officerallegation
                    WHERE data_officerallegation.allegation_category_id IN
                      (SELECT id
                      FROM data_allegationcategory
                      WHERE data_allegationcategory.category = 'Drug / Alcohol Abuse' OR data_allegationcategory.category = 'Medical' or allegation_name LIKE 'Medical Roll%'
                      OR data_allegationcategory.category_code IN ('024', '003', '003A', '003B', '003C', '003D', '003E'))))
    group by officer_id);


select withoutRelation.id,withoutRelation.officer_name,award_count.num_awards from withoutRelation
join award_count on withoutRelation.id=award_count.offid;


'''

In [ ]:
import pandas as pd

In [ ]:
cursor.execute(edges_query)
edges = cursor.fetchall()
print("shape is: " + str(len(edges))) # 17465

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames

print(df_edges)

shape is: 17399
         src    dst  relationship
0          9  12641             1
1         17  26456             2
2         17  10656             2
3         17  22523             1
4         17  20246             1
...      ...    ...           ...
17394  32289  32405             1
17395  32289  32431             1
17396  32397  32419             1
17397  32397  32406             1
17398  32527  33722             2

[17399 rows x 3 columns]


In [ ]:
cursor.execute(nodes_query)
nodes = cursor.fetchall()
print("shape is: " + str(len(nodes))) # 17465

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames

print(df_nodes)

shape is: 204
        id          officer_name  num_awards
0     1907        William Bertha          12
1     2735  Charles Breckenridge           6
2     2941         Deborah Brown           1
3     3087           Henry Brown          12
4     3927         Glenn Carlson           2
..     ...                   ...         ...
199  30652      Edward Wilkowski          10
200  30653            Alan Wilks           1
201  30952            Sam Wilson          10
202  31464         Michael Ytsen          15
203  32188       Elmore Metcalfe          17

[204 rows x 3 columns]


In [ ]:
edges_ = spark.createDataFrame(df_edges)

In [ ]:
nodes = spark.createDataFrame(df_nodes)

In [ ]:
cpdb = GraphFrame(nodes, edges_)

In [ ]:
cpdb.vertices.show()

+----+--------------------+----------+
|  id|        officer_name|num_awards|
+----+--------------------+----------+
|1907|      William Bertha|        12|
|2735|Charles Breckenridge|         6|
|2941|       Deborah Brown|         1|
|3087|         Henry Brown|        12|
|3927|       Glenn Carlson|         2|
|4233|      John Catanzara|        12|
|4249|     Alonzo Caudillo|         1|
|4554|       Walter Chodor|         1|
|4715|      William Clancy|        13|
|4880|          Shawn Cobb|        12|
|4881|        Gilbert Cobb|         3|
|5170|    Gregory Connelly|        18|
|5404|      George Cornish|         5|
|5513|        James Cotton|         6|
|5683|        Victor Creed|         1|
|5820|          Jimmy Cruz|         1|
|6276|       Quinton Davis|        12|
|6400|        Quadir Dawan|        20|
|6618|     Aaron Del Valle|        15|
|6996|       Vincent Dixon|        17|
+----+--------------------+----------+
only showing top 20 rows



In [ ]:
cpdb.edges.show()

+---+-----+------------+
|src|  dst|relationship|
+---+-----+------------+
|  9|12641|           1|
| 17|26456|           2|
| 17|10656|           2|
| 17|22523|           1|
| 17|20246|           1|
| 17|15351|           1|
| 17|12521|           1|
| 17| 9373|           1|
| 17| 6885|           1|
| 17| 4494|           1|
| 17|32102|           1|
| 17|28925|           1|
| 17|27976|           1|
| 54|27802|           1|
| 54|29189|           1|
| 54|30398|           1|
| 54|31550|           1|
| 54|31756|           1|
| 54| 4168|           1|
| 54|11252|           1|
+---+-----+------------+
only showing top 20 rows



In [ ]:
cpdb.vertices.sort(['id'],ascending=True).show()

+----+--------------------+----------+
|  id|        officer_name|num_awards|
+----+--------------------+----------+
|1907|      William Bertha|        12|
|2735|Charles Breckenridge|         6|
|2941|       Deborah Brown|         1|
|3087|         Henry Brown|        12|
|3927|       Glenn Carlson|         2|
|4233|      John Catanzara|        12|
|4249|     Alonzo Caudillo|         1|
|4554|       Walter Chodor|         1|
|4715|      William Clancy|        13|
|4880|          Shawn Cobb|        12|
|4881|        Gilbert Cobb|         3|
|5170|    Gregory Connelly|        18|
|5404|      George Cornish|         5|
|5513|        James Cotton|         6|
|5683|        Victor Creed|         1|
|5820|          Jimmy Cruz|         1|
|6276|       Quinton Davis|        12|
|6400|        Quadir Dawan|        20|
|6618|     Aaron Del Valle|        15|
|6996|       Vincent Dixon|        17|
+----+--------------------+----------+
only showing top 20 rows



In [ ]:
pr_cpdb = cpdb.pageRank(resetProbability=0.15, tol=0.01)
#look at the pagerank score for every vertex
pr_cpdb.vertices.orderBy('pagerank', ascending=False).show()
pr_cpdb.edges.show()

+-----+-------------------+----------+------------------+
|   id|       officer_name|num_awards|          pagerank|
+-----+-------------------+----------+------------------+
|31464|      Michael Ytsen|        15|2.0803279453002217|
|17964|  Michael Mc Carthy|        13| 1.910639575934099|
|27442|      Scott Stehlik|        11|1.8928392785739436|
|18359|     John Mc Knight|        13| 1.804237365395038|
|12641|     Charles Howard|        13|1.7209128686438844|
| 9713|         Gary Gaski|         1|1.6844792794109509|
|29668|      Dean Vitulski|        15|1.6721654236222592|
|23410|      Raphael Ramos|        15| 1.667130229111527|
| 7597|     Patrick Durkin|        19|1.6447051684758143|
|29056|     Charles Turner|         1| 1.633025278576153|
|13270|    Jason Jankowski|        19| 1.633025278576153|
|30313|        George Weir|        14| 1.633025278576153|
|28580|      Edward Tiedje|         3| 1.633025278576153|
|22751|Sterling Poindexter|         1| 1.633025278576153|
|19750|       